# Video Quality Analysis

Video players / streaming platforms incur costs based on MB/s delivered. This varies when a user plays a video at specific quality settings (360p, 480p, ... , 1080p HD). The goal of this analysis is to compare a specific daterange of user viewership data and see the cost difference. The analysis compares the original viewership costs, versus an "Adaptive" setting - meaning that while default is 360p, if a user changes to a higher setting, the player will remember that setting for the next time, staying there until the user changes to a higher setting again.

In [5]:
import pandas as pd
import numpy as np
from funcy import flatten

Import data from csv. CSV data not shown to protect company privacy.

In [ ]:
Location = 'sampledata.csv'
csv_data = pd.read_csv(Location)
csv_data.columns = ['userid', 'os', 'quality_status', 'total_seconds', 'distinct_id', 'video_content', 'time']

Only looking at desktop data

In [90]:
mac_data = csv_data[csv_data.os == 'Mac OS X']
windows_data = csv_data[csv_data.os == 'Windows']
linux_data = csv_data[csv_data.os == 'Linux']
os_data = pd.concat([mac_data, windows_data, linux_data], axis=0)

In [91]:
interval_data = os_data.sort(['distinct_id', 'time'])
interval_data.index = range(0,len(interval_data))
unique_interval_distinct_id = pd.unique(interval_data.distinct_id.ravel())
qualities = pd.unique(interval_data.quality_status.ravel())

array(['480p-30', '720p-60', 'DQ-DFR'], dtype=object)

In [93]:
qualities_level = array( [25.95, 4.47, 9.96, 25.95, 5.84, 25.95, 25.95, 4.47, 25.95, 9.96, 5.84, 9.96, 25.95, 9.96, 9.96, 25.95] )
qualities_values = np.vstack((qualities,qualities_level)).T
qualities_values = pd.DataFrame(qualities_values)

Sort through data, change accordingly based on video setting.

In [97]:
test = []
total_adaptive = []
for i in range(0, len(qualities)):
    adaptive_indiv_br = []
    if min(numpy.where(interval_data['distinct_id'] == unique_interval_distinct_id[i])[0]) == max(numpy.where(interval_data['distinct_id'] == unique_interval_distinct_id[i])[0]):
        adaptive_indiv_br = qualities_level[(pd.match([(interval_data['quality_status'][i])],qualities))]
    else:
        adaptive_indiv_br = qualities_level[(pd.match((interval_data['quality_status'][(min(numpy.where(interval_data['distinct_id'] == unique_interval_distinct_id[i])[0])):((max(numpy.where(interval_data['distinct_id'] == unique_interval_distinct_id[i])[0])) + 1)]),qualities))]
    test.append(adaptive_indiv_br)

    new_values = []
    new_values.append(test[i][0])
    for x in range(1,len(adaptive_indiv_br)):
        if adaptive_indiv_br[x] > max(adaptive_indiv_br[0:x]):
            new_values.append(adaptive_indiv_br[x])
        else:
            new_values.append(max(adaptive_indiv_br[0:x]))
    total_adaptive.append(new_values)

find total original MB/s delivered

In [8]:
total_adaptive = flatten(total_adaptive)
total_adaptive = numpy.round(total_adaptive,2)
total_actual = []
for i in range(0,len(test)):
    total_actual.append(sum(test[i]))
total_actual = numpy.round(total_actual,2)
sum(total_actual)

find difference in costs, percentage

In [7]:
(sum(total_adaptive) - sum(total_actual)) / sum(total_actual)